# Filter data to predict the individual transfer stop and visualization

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import dateutil
import re
import folium
import os
from math import radians, cos, sin, asin, sqrt

## Function part
### We define multiple function to deal with our data.
#### Transit run on different routes at different times of the year, so we classify our GTFS data according to different times and store them into different files.

In [2]:
# Choose which time period of GTFS data to use.
def GTFS_table(a,file):
    # USe strptime function to convert the time formate.
    if datetime.strptime(a, '%Y-%m-%d  %H:%M:%S') <= datetime(2016,6,10,0,0):
        return pd.read_csv("data/3-6/"+ file)
    elif datetime.strptime(a, '%Y-%m-%d  %H:%M:%S') <= datetime(2016,8,19,0,0):
        return pd.read_csv("data/6-8/"+ file)
    elif datetime.strptime(a, '%Y-%m-%d  %H:%M:%S') <= datetime(2016,12,2,0,0):
        return pd.read_csv("data/8-12/"+ file)
    else:
        return pd.read_csv("data/12-3/"+ file)

Usually the service_id that has been splited has five strings, which means that this line runs every day from Monday to Friday; If there are a total of six strings for service_id that has been splited, it means that the running time of this line is not regular and may only run one day per week. At the same time, the sixth string also indicates which days of the week transit is operating. If the sixth string is '0010001', it means the line is operating only on Wednesdays and Sundays.

In [3]:
# Split the "servise_id" field to multiple strings and get the nth string by using index
def split_n(a,n):
    if n == 5:
        if len(a.split('-')) < 6:
            return 'NULL'
        else: return a.split('-')[n]
    else: return a.split('-')[n]

In [4]:
#The service time of each transit is extracted by the sixth string of the splited service_id.
# Change the format of the sixth string from 0 or 1 to 0-6 to indicate the actual running time of transit.
def week_day(a):
    w = [0,1,2,3,4,5,6]  #Monday - Sunday
    a = list(a)
    WeekList = []
    for i in range(len(a)):
        if a[i] == '1':
            WeekList.append(w[i])
    return WeekList

In [5]:
# The function to calcualte distance between bording coordinate and stop coordinate
def haversine(row,place, index):
    # convert decimal degrees to radians 
    if place == 'ORIGIN':
        lon1, lat1, lon2, lat2 = map(radians, [row["stop_lon"], row["stop_lat"], on_board.iat[index,10], on_board.iat[index,9]])
    elif place == 'BOARDING':
        lon1, lat1, lon2, lat2 = map(radians, [row["stop_lon"], row["stop_lat"], on_board.iat[index,7], on_board.iat[index,6]])
    # haversine formula 
    dlon = abs(lon2 - lon1) 
    dlat = abs(lat2 - lat1) 
    a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2)**2)
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [6]:
#The function to find the Nth smallest number of our distance field
def smallest_n(a,order):
    a = list(set(a))
    a.sort()
    return a[order]

#### Extract the individual travel time (hour) in on_board data

In [7]:
# Read the csv file into the pandas dataframe
on_board = pd.read_csv("data/on_board.csv")
# Take the first eleven individual as the example
on_board.iloc[0:11].to_csv('data/on_board1.csv')
on_board = pd.read_csv('data/on_board1.csv')

p1 = re.compile(r"(\w+):.*") # Create pattern object time
p2 = re.compile(r".*([a-z][a-z])") # Create pattern object am/pm

# Extract the hour part and am/pm and add with two columns
on_board["Time"], on_board["Clock"] = on_board.TIME_ON.str.extract(p1), on_board.TIME_ON.str.extract(p2)
on_board["Time"] = on_board["Time"].astype(int)

# Convert the time formate
on_board.Time = np.where((on_board.Clock == "pm") & (on_board.Time !=12) & (on_board.Time !=11), on_board.Time + 12, on_board.Time)
on_board["Time"] = on_board["Time"].astype(int)
on_board["route_type"] = on_board["route_type"].astype(str)
# Change the format of route——type in the onboard table to be consistent with the format within GTFS data 
on_board["route_type"] = on_board["route_type"].replace({'ALINE':'A Line', 'BLUE':'Blue', 'GREEN':'Green'})
on_board

,Unnamed: 0,Unnamed: 0.1,ID,DATE,ROUTE_SURVEYED,BOARDING_LOCATION,BOARDING_LAT,BOARDING_LON,TRANSFERS_FROM_FIRST,ORIGIN_LAT_100M,ORIGIN_LON_100M,TIME_ON,route_type,Vertical_direction,Horizontal_direction,Time,Clock
0,0,0,1,2016-04-11 00:00:00,BLUE METRO Blue Line [SB],Warehouse Hennepin Ave Station,44.980003,-93.273144,6 U of M - Hennepin - Xerxes - France - Southd...,44.948,-93.299,12:00 pm - 1:00 pm,6,0.032003,0.025856,12,pm
1,1,1,2,2016-04-11 00:00:00,BLUE METRO Blue Line [SB],38th St Station,44.934613,-93.229421,22 Brklyn Ctr - Lyndale Av N - Cedar - 28th Av...,44.955,-93.246,12:00 pm - 1:00 pm,22,-0.020387,0.016579,12,pm
2,2,3,4,2016-04-11 00:00:00,BLUE METRO Blue Line [NB],MOA Transit Station,44.854287,-93.238861,RED METRO Red Line [ Minnesota Valley Transit ],44.830,-93.153,4:00 - 5:00 pm,RED,0.024287,-0.085861,16,pm
3,3,7,8,2016-04-12 00:00:00,GREEN METRO Green Line [WB],Capitol / Rice St Station,44.955749,-93.105534,62 Rice St - Little Canada - Shoreview - Signa...,45.107,-93.127,3:00 - 4:00 pm,62,-0.151251,0.021466,15,pm
4,4,10,11,2016-04-12 00:00:00,GREEN METRO Green Line [EB],Nicollet Mall Station,44.978491,-93.269912,18 Nicollet Av - South Bloomington [ Minneapol...,44.947,-93.278,4:00 - 5:00 pm,18,0.031491,0.008088,16,pm
5,5,13,14,2016-04-12 00:00:00,GREEN METRO Green Line [EB],Dale St Station,44.955682,-93.125945,65 Dale St - Co Rd B - Rosedale [ Minneapolis ...,45.006,-93.145,4:00 - 5:00 pm,65,-0.050318,0.019055,16,pm
6,6,18,19,2016-04-12 00:00:00,GREEN METRO Green Line [WB],Central Station,44.946070,-93.092202,54 Ltd Stop - W 7St - Airport - MOA [ Minneapo...,44.874,-93.228,8:00 - 9:00 pm,54,0.072070,0.135798,20,pm
7,7,19,20,2016-04-13 00:00:00,21 Uptown - Lake St - Selby Av [EB],Uptown Station Leave & Gates D/E,44.950073,-93.297885,6 U of M - Hennepin - Xerxes - France - Southd...,44.922,-93.327,7:00 - 8:00 am,6,0.028073,0.029115,7,am
8,8,20,21,2016-04-13 00:00:00,21 Uptown - Lake St - Selby Av [EB],Uptown Station & Terminal,44.950079,-93.297545,12 Uptown - Excelsior Blvd - Hopkins - Opus [ ...,44.938,-93.332,7:00 - 8:00 am,12,0.012079,0.034455,7,am
9,9,21,22,2016-04-13 00:00:00,21 Uptown - Lake St - Selby Av [EB],Chicago Lake Transit Center & Gate A,44.949296,-93.262280,5 Brklyn Center - Fremont - 26th Av - Chicago ...,45.034,-93.295,8:00 - 9:00 am,5,-0.084704,0.032720,8,am


## Start filter data to find the most likely transfer station
#### Call the GTFS_table function, and use the "Date" field in the onboard table to determine using which GTFS data.

In [8]:
# Create a list to contain the temp result of each individual
dfs = []

# Use for loop to iterate the on_board table
for index1, row1 in on_board.iterrows():

#Start filter data to find the most likely transfer station
#Call the GTFS_table function, and use the "Date" field in the onboard table to determine using which GTFS data.

    # Read the csv file into the pandas dataframe
    temp = GTFS_table(on_board.iat[index1,3],'merge_extract.csv')
    
    temp['route_short_name'] = temp['route_short_name'].astype(str)   #change to string type
    
#Remove the weekend data in GTFS and filter the route name
#Look for the route number that matches the first passenger transfer line and 
#remove the data in the GTFS table that the service time belongs to weekend, 
#because the on_board table does not contain weekend data.

    temp = temp[temp["route_short_name"] == on_board.iat[index1,12]]   #filter route name (e.g.route 6)
# GTFS data do not onclude Red Line, so we use condition statement to skip the individual who took the Red Line
    if len(temp) == 0:
        continue
    temp = temp[(temp['service_id'].apply(split_n,args =[3]) == 'Weekday') 
            | (temp['service_id'].apply(split_n,args =[3]) == 'Holiday')] #filter and save weekday/holiday data

#Filter the service day
#Each route will run on a different line depending on the date, 
#we will only select the row that the start date is smaller than the first passenger transfer date and
#the ending data is larger than the transfer date.      
    
    temp['start_date'] = temp['start_date'].astype(str)   #change to string type
    temp['end_date'] = temp['end_date'].astype(str)
    temp = temp[(datetime.strptime(on_board.iat[index1,3], '%Y-%m-%d  %H:%M:%S') 
             >= temp['start_date'].apply(datetime.strptime,args =['%Y%m%d'])) 
            & (datetime.strptime(on_board.iat[index1,3], '%Y-%m-%d  %H:%M:%S') 
               <= temp['end_date'].apply(datetime.strptime,args =['%Y%m%d']))]

#Filter exception date
#Sometimes the line is run throughout weekday, but may be shut down one day due to an emergency, 
#so special cases like this need to be considered. In GTFS ' exception data, if 'exception type' equals 2, 
#means that the line stops running on the date of exception.

    exception = GTFS_table(on_board.iat[index1,3],'calendar_dates.txt') #read exception date
    exception = exception[exception.exception_type == 2]  #2 means it should be removed
    exception['date'] = exception['date'].astype(str)   #change to string type

    # Find out if any bus lines stopped running on the day the first passenger traveled.
    exception = exception[datetime.strptime(on_board.iat[index1,3], '%Y-%m-%d  %H:%M:%S')
                      == exception['date'].apply(datetime.strptime,args =['%Y%m%d'])]

    # If find a trip that stops running on that day, remove it.
    if len(exception) >0:
        temp = temp[temp['service_id'].isin(list(exception['service_id'])) == False]  #remove exception service  

#Match the day of a week
#Some transit trips run only on Mondays, and if the first individual travels on a Tuesday, the line must be excluded. 
#In the code below, we compare the day of weeks a passenger travels with the GTFS data.

    # Filter the trips that don't serve for all weekday
    temp_week = temp[temp['service_id'].apply(split_n,args =[5]) !='NULL']

    # Use 'week_day' function to convert the formate
    if len(temp_week) > 0:
        temp_week['weekday'] = temp_week['service_id'].apply(split_n,args =[5]) #get 6th string
        temp_week['weekday'] = temp_week['weekday'].apply(week_day) #get week list
    
    # Match the day of a week
        temp_week = temp_week[temp_week['weekday'].apply(
            lambda x: datetime.strptime(on_board.iat[index1,3], '%Y-%m-%d  %H:%M:%S').weekday() not in x)]
    
    # Remove the unmatch data 
        temp = temp[temp['service_id'].isin(list(temp_week['service_id'])) == False]

#Merge stop data to 'temp' file and slice fields
#In this section, we will match the stop time with individuals' bording time.
        
    stop_times = GTFS_table(on_board.iat[index1,3],'stop_times.txt')  # Read the stop times file
    stops = GTFS_table(on_board.iat[index1,3],'stops.txt')  # Read the stops file

#Filter the stop time with boarding time
#Since the accuracy of individual travel time in on_board data is one hour, we can only compare the the hour part of travel time and stop time. 
#And the time in GTFS is expressed in the 24-hour system, 
#while the time in on_board data is expressed in the 12-hour system, so we need to convert it.
    
    # Extract the arrive time (hour) in GTFS data    
    p = re.compile(r"(\w+):.*")  # Create a pattern object time
    stop_times["Time"] = stop_times["arrival_time"].str.extract(p)  # Extract the hour part and add as a new column
    
    # Merge temp file with stop times file based on the column "trip_id"
    merge_1 = pd.merge(temp, stop_times, on ="trip_id", how = "left" )
    # Merge stops file with previous merged dataframe
    merge_2 = pd.merge(merge_1, stops, on = "stop_id", how = "left")
    # Slice out the needed columns
    temp = merge_2.loc[:, 
                            ["route_id" , "route_short_name" , 
                            "route_long_name" , "trip_id" , 
                            "service_id", "trip_headsign", 
                            "direction_id" ,
                            "arrival_time" , "stop_id", 
                             "stop_sequence", "stop_name", 
                            "stop_lat" , "stop_lon", "start_date", "end_date", "Time"]
                      ]
    
    temp["Time"] = temp["Time"].astype(int)
    temp = temp[temp.Time < 24]  # Drop the rows having the hour value larger than 24

    #filter the same time period in these two data
    temp = temp[temp["Time"] == on_board.iat[index1,-2]] 

    #Calcualte the distance between individual bording location and stop location   
    temp['Dist'] = temp.apply(haversine,args = ('BOARDING', index1),axis=1)

#Choose the stop that has the smallest disctance and filter the direction
#The general idea is to first screen out the station nearest to the next transit boarding place, t
#hen filter the station nearest to origin place based on these records, 
#and then check whether the direction of trip is correct. 
#We check the direction through the "stop_sequence" field, 
#which encodes the station in the order from the originating station to the destination station. 
#The "stop_sequence" for the latter station is always larger than the previous station.
    
    #initialization
    temp_stop = temp[temp["Dist"] == temp["Dist"].min()]
    initial = 0
    order = 0 # The index of the nth nearest stop
    
    #If it is found that there is no matching record based on the station nearest to the next transit boarding location,
    #then the station with the second smallest distance shall be selected for the next wave of less selection judgment'''
    
    while(len(temp_stop) == 0 or initial == 0): 
        temp_stop = temp[temp["Dist"] == smallest_n(temp["Dist"],order)]  # The Nth nearest stop
    
    # Assign value to the origin place based on the trip_id in temp_stop data frame.
        temp_origin = temp[temp['trip_id'].isin(list(temp_stop['trip_id']))]
    
    # Calcualte the distance between stop and origin location and find the shortest one.
        temp_origin['Distance'] = temp_origin.apply(haversine,args =('ORIGIN', index1),axis=1)
        temp_origin = temp_origin[temp_origin["Distance"] == temp_origin["Distance"].min()]
    
    # Filter the temp_stop data based on the shortest distance between stop and origin location
        temp_stop = temp_stop[temp_stop['trip_id'].isin(list(temp_origin['trip_id']))]
    
    # Use sequence field to check the direction
        temp_origin = temp_origin[['trip_id','stop_id','stop_name','stop_lon','stop_lat','stop_sequence']]
        temp_stop = pd.merge(temp_stop, temp_origin, on ="trip_id", how = "left" )  #merge temp_origin and temp_stop
    
        temp_stop = temp_stop[temp_stop['stop_sequence_x'] > temp_stop['stop_sequence_y']]  #check the sequence
        order = order + 1
        initial = 1
    
    # Create a new colum and assign the individual ID to it
    temp_stop['indivi_id'] = on_board.iat[index1,2]
    # Append each individual's temp table to the dfs list
    dfs.append(temp_stop)
# Conver the list to a new dataframe called 'final'
final = pd.concat(dfs)
final.head()

d:\Users\yaxua\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
d:\Users\yaxua\Anaconda3\lib\site-packages\ipykernel_launcher.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,route_id,route_short_name,route_long_name,trip_id,service_id,trip_headsign,direction_id,arrival_time,stop_id_x,stop_sequence_x,...,start_date,end_date,Time,Dist,stop_id_y,stop_name_y,stop_lon_y,stop_lat_y,stop_sequence_y,indivi_id
0,6-81,6,U of M - Hennepin - Xerxes - France - Southdale,9181751-MAR16-MVS-BUS-Weekday-01,MAR16-MVS-BUS-Weekday-01,Northbound 6 Downtown / Via Xerxes,0,12:29:00,17947,77,...,20160321,20160610,12,0.06877,1322,Hennepin Ave & Lake St W,-93.298208,44.948272,60,1
1,6-81,6,U of M - Hennepin - Xerxes - France - Southdale,9181753-MAR16-MVS-BUS-Weekday-01,MAR16-MVS-BUS-Weekday-01,Northbound 6 Downtown / Via Xerxes,0,12:49:00,17947,77,...,20160321,20160610,12,0.06877,1322,Hennepin Ave & Lake St W,-93.298208,44.948272,60,1
2,6-81,6,U of M - Hennepin - Xerxes - France - Southdale,9181763-MAR16-MVS-BUS-Weekday-01,MAR16-MVS-BUS-Weekday-01,Northbound 6U 27Av-Univ / Via France,0,12:19:00,17947,62,...,20160321,20160610,12,0.06877,1322,Hennepin Ave & Lake St W,-93.298208,44.948272,45,1
3,6-81,6,U of M - Hennepin - Xerxes - France - Southdale,9181764-MAR16-MVS-BUS-Weekday-01,MAR16-MVS-BUS-Weekday-01,Northbound 6U 27Av-Univ / Via France,0,12:39:00,17947,62,...,20160321,20160610,12,0.06877,1322,Hennepin Ave & Lake St W,-93.298208,44.948272,45,1
4,6-81,6,U of M - Hennepin - Xerxes - France - Southdale,9181769-MAR16-MVS-BUS-Weekday-01,MAR16-MVS-BUS-Weekday-01,Northbound 6U 27Av-Univ / Via Wooddale,0,12:59:00,17947,66,...,20160321,20160610,12,0.06877,1322,Hennepin Ave & Lake St W,-93.298208,44.948272,49,1


In [9]:
# Slice the required column of GTFS and on_board data
final = final.loc[:, 
                            ["indivi_id","route_id","route_short_name","trip_id","stop_lat_x","stop_lon_x","stop_lat_y",
                             "stop_lon_y","stop_id_x","stop_id_y", "arrival_time","stop_name_x","stop_name_y"]
                     ]

on_board = on_board.loc[:, 
                            ["ID","DATE","BOARDING_LON","BOARDING_LAT","ORIGIN_LAT_100M","ORIGIN_LON_100M"]
                     ]

# Merge the two table
result = pd.merge(final, on_board, left_on ="indivi_id", right_on = "ID", how = "left" )

result

,indivi_id,route_id,route_short_name,trip_id,stop_lat_x,stop_lon_x,stop_lat_y,stop_lon_y,stop_id_x,stop_id_y,arrival_time,stop_name_x,stop_name_y,ID,DATE,BOARDING_LON,BOARDING_LAT,ORIGIN_LAT_100M,ORIGIN_LON_100M
0,1,6-81,6,9181751-MAR16-MVS-BUS-Weekday-01,44.979544,-93.272558,44.948272,-93.298208,17947,1322,12:29:00,Hennepin Ave & 5th St S,Hennepin Ave & Lake St W,1,2016-04-11 00:00:00,-93.273144,44.980003,44.948,-93.299
1,1,6-81,6,9181753-MAR16-MVS-BUS-Weekday-01,44.979544,-93.272558,44.948272,-93.298208,17947,1322,12:49:00,Hennepin Ave & 5th St S,Hennepin Ave & Lake St W,1,2016-04-11 00:00:00,-93.273144,44.980003,44.948,-93.299
2,1,6-81,6,9181763-MAR16-MVS-BUS-Weekday-01,44.979544,-93.272558,44.948272,-93.298208,17947,1322,12:19:00,Hennepin Ave & 5th St S,Hennepin Ave & Lake St W,1,2016-04-11 00:00:00,-93.273144,44.980003,44.948,-93.299
3,1,6-81,6,9181764-MAR16-MVS-BUS-Weekday-01,44.979544,-93.272558,44.948272,-93.298208,17947,1322,12:39:00,Hennepin Ave & 5th St S,Hennepin Ave & Lake St W,1,2016-04-11 00:00:00,-93.273144,44.980003,44.948,-93.299
4,1,6-81,6,9181769-MAR16-MVS-BUS-Weekday-01,44.979544,-93.272558,44.948272,-93.298208,17947,1322,12:59:00,Hennepin Ave & 5th St S,Hennepin Ave & Lake St W,1,2016-04-11 00:00:00,-93.273144,44.980003,44.948,-93.299
5,2,22-81,22,9194819-MAR16-MVS-BUS-Weekday-01,44.934870,-93.229731,44.955374,-93.247437,51543,15368,12:22:00,38th St Station & Gate B,Cedar Ave S & 26th St E,2,2016-04-11 00:00:00,-93.229421,44.934613,44.955,-93.246
6,2,22-81,22,9194827-MAR16-MVS-BUS-Weekday-01,44.934870,-93.229731,44.955374,-93.247437,51543,15368,12:42:00,38th St Station & Gate B,Cedar Ave S & 26th St E,2,2016-04-11 00:00:00,-93.229421,44.934613,44.955,-93.246
7,8,62-81,62,9046097-MAR16-MVS-BUS-Weekday-01,44.955960,-93.106029,45.086444,-93.126515,975,46951,15:44:00,Rice St & University Ave,Tanglewood Dr & Laura Lane,8,2016-04-12 00:00:00,-93.105534,44.955749,45.107,-93.127
8,11,18-81,18,9185446-MAR16-MVS-BUS-Weekday-01,44.979544,-93.272558,44.946543,-93.277857,17947,1887,16:42:00,Hennepin Ave & 5th St S,Nicollet Ave S & 31st St E,11,2016-04-12 00:00:00,-93.269912,44.978491,44.947,-93.278
9,11,18-81,18,9185479-MAR16-MVS-BUS-Weekday-01,44.979544,-93.272558,44.946543,-93.277857,17947,1887,16:57:00,Hennepin Ave & 5th St S,Nicollet Ave S & 31st St E,11,2016-04-12 00:00:00,-93.269912,44.978491,44.947,-93.278


In [10]:
# Group the 'trip_id' column of each individual into a list.
temp1 = result.groupby('indivi_id')['trip_id'].apply(','.join).reset_index()
# Group the 'arrive_time' column of each individual into a list.
temp2 = result.groupby('indivi_id')['arrival_time'].apply(','.join).reset_index()
# Drop the 'trip_id' and 'arrival_time' columns from the merge table.
temp3 = result.drop(['trip_id', 'arrival_time', 'ID'], axis=1)
# Drop the duplicates row
temp3 = temp3.drop_duplicates()
# Merge three temp tabel together
merge_1 = pd.merge(temp3, temp1, on ="indivi_id", how = "left" )
final_result = pd.merge(merge_1, temp2, on ="indivi_id", how = "left" )

final_result

,indivi_id,route_id,route_short_name,stop_lat_x,stop_lon_x,stop_lat_y,stop_lon_y,stop_id_x,stop_id_y,stop_name_x,stop_name_y,DATE,BOARDING_LON,BOARDING_LAT,ORIGIN_LAT_100M,ORIGIN_LON_100M,trip_id,arrival_time
0,1,6-81,6,44.979544,-93.272558,44.948272,-93.298208,17947,1322,Hennepin Ave & 5th St S,Hennepin Ave & Lake St W,2016-04-11 00:00:00,-93.273144,44.980003,44.948,-93.299,"9181751-MAR16-MVS-BUS-Weekday-01,9181753-MAR16...","12:29:00,12:49:00,12:19:00,12:39:00,12:59:00"
1,2,22-81,22,44.934870,-93.229731,44.955374,-93.247437,51543,15368,38th St Station & Gate B,Cedar Ave S & 26th St E,2016-04-11 00:00:00,-93.229421,44.934613,44.955,-93.246,"9194819-MAR16-MVS-BUS-Weekday-01,9194827-MAR16...","12:22:00,12:42:00"
2,8,62-81,62,44.955960,-93.106029,45.086444,-93.126515,975,46951,Rice St & University Ave,Tanglewood Dr & Laura Lane,2016-04-12 00:00:00,-93.105534,44.955749,45.107,-93.127,9046097-MAR16-MVS-BUS-Weekday-01,15:44:00
3,11,18-81,18,44.979544,-93.272558,44.946543,-93.277857,17947,1887,Hennepin Ave & 5th St S,Nicollet Ave S & 31st St E,2016-04-12 00:00:00,-93.269912,44.978491,44.947,-93.278,"9185446-MAR16-MVS-BUS-Weekday-01,9185479-MAR16...","16:42:00,16:57:00,16:29:00,16:49:00,16:35:00"
4,14,65-81,65,44.955873,-93.126444,45.006368,-93.146536,10746,3967,Dale St & University Ave,Co Rd B & Lexington Ave,2016-04-12 00:00:00,-93.125945,44.955682,45.006,-93.145,"9231583-MAR16-MVS-BUS-Weekday-01,9231597-MAR16...","16:48:00,16:28:00"
5,19,54-81,54,44.946915,-93.091744,44.860024,-93.222824,49397,41975,5th St & Minnesota St,34th Ave S & American Blvd / I-494,2016-04-12 00:00:00,-93.092202,44.946070,44.874,-93.228,"9050257-MAR16-MVS-BUS-Weekday-01,9050304-MAR16...","20:34:00,20:49:00"
6,20,6-81,6,44.950151,-93.298210,44.921891,-93.329026,50195,1292,Hennepin Ave & Uptown Station - Gate B,France Ave S & Sunnyside Ave,2016-04-13 00:00:00,-93.297885,44.950073,44.922,-93.327,"9181582-MAR16-MVS-BUS-Weekday-01,9181584-MAR16...","07:15:00,07:26:00,07:41:00,07:50:00"
7,21,12-81,12,44.950151,-93.298210,44.937982,-93.332069,50195,4856,Hennepin Ave & Uptown Station - Gate B,Excelsior Blvd & Huntington Ave,2016-04-13 00:00:00,-93.297545,44.950079,44.938,-93.332,"9047419-MAR16-MVS-BUS-Weekday-01,9047438-MAR16...","07:52:00,07:35:00,07:21:00"
8,22,5-81,5,44.949296,-93.262280,45.033572,-93.295826,52261,9609,Chicago Lake Transit Center & Gate A,Fremont Ave N & 43rd Ave N,2016-04-13 00:00:00,-93.262280,44.949296,45.034,-93.295,9167304-MAR16-MVS-BUS-Weekday-01,08:53:00
9,25,87-81,87,44.948455,-93.187430,44.954280,-93.182371,18903,18894,Cleveland Ave & Marshall Ave,Prior Ave & Feronia Ave,2016-04-13 00:00:00,-93.187244,44.948417,44.954,-93.179,"9045208-MAR16-MVS-BUS-Weekday-01,9045226-MAR16...","09:07:00,09:27:00,09:47:00"


In [11]:
# Save the result to csv table
final_result.to_csv("data/individual_10.csv")